In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/ColabNotebooks/BirdCLEF2024

/content/drive/MyDrive/ColabNotebooks/BirdCLEF2024


In [3]:
!cp /content/drive/MyDrive/ColabNotebooks/BirdCLEF2024/data/birdclef-2024.zip /content

In [4]:
%cd /content

/content


In [5]:
!unzip -q birdclef-2024.zip

In [6]:
import cv2
import audioread
import logging
import gc
import os
import sys
import random
import time
import glob
import warnings
warnings.simplefilter('ignore')
from tqdm import tqdm

import librosa
import librosa.display
import IPython.display as ipd
import numpy as np
import pandas as pd
import soundfile as sf
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as torchdata
import torchaudio
import torchaudio.transforms as T

from contextlib import contextmanager
from joblib import Parallel, delayed
from pathlib import Path
from typing import Optional
from sklearn.model_selection import StratifiedKFold, GroupKFold

from albumentations.core.transforms_interface import ImageOnlyTransform
import albumentations as A
import albumentations.pytorch.transforms as T
from PIL import Image

import matplotlib.pyplot as plt

In [7]:
from socket import SOCK_RDM
SR = 32000
USE_SEC = 5 # 60 # 90 # 60
TH_SEC = 1

def Audio_to_Array(path):
    y, sr = sf.read(path, always_2d=True)
    y = np.mean(y, 1) # there is (X, 2) array
    if len(y) > SR:
        y = y[SR:-SR]
        print(y.shape)

    if len(y) > SR * USE_SEC:
        y = y[:SR * USE_SEC]
    return y

def Audio_to_Array_full(path):
    y, sr = sf.read(path, always_2d=True)
    y = np.mean(y, 1) # there is (X, 2) array
    n_period = int(y.shape[0]/(SR*USE_SEC))
    waves = []
    if len(y) < SR*TH_SEC:
        waves = None
    elif n_period==0:
        waves.append(y)
    else:
        for i in range(n_period):
            waves.append(y[i*(SR*USE_SEC):(i+1)*(SR*USE_SEC)])
    return waves, sr


def save_full(path):
    waves = Audio_to_Array_full(path)
    file_paths = []
    if waves is None:
        pass
    else:
        folder, filename_org = path.split('/')[-2:]
        for i, wave in enumerate(waves):
            filename = filename_org.split('.')[0] + f'_{i}'
            file_path = folder + "/" + filename
            save_path = "./train_audio_sep_5sec_ogg/" + file_path
            np.save(save_path, wave)
            file_paths.append(file_path)
    return file_paths

def save_ogg_full(path):
    waves, sr = Audio_to_Array_full(path)
    file_paths = []
    if waves is None:
        pass
    else:
        folder, filename_org = path.split('/')[-2:]
        for i, wave in enumerate(waves):
            filename = filename_org.split('.')[0] + f'_{i}.ogg'
            file_path = folder + "/" + filename
            save_path = "/content/train_audio_sep_5sec_ogg/" + file_path
            sf.write(save_path, wave, sr)
            #print(save_path)
            file_paths.append(file_path)
    return file_paths

def make_clip_df(record, clip_record):
    df = pd.DataFrame()
    for i in range(len(clip_record)):
        df = df.append(record, ignore_index=True)
    df['clip_filename'] = clip_record
    return df

AUDIO_PATH = './train_audio'


train = pd.read_csv('./train_metadata.csv')
train["file_path"] = AUDIO_PATH + '/' + train['filename']
paths = train["file_path"].values

In [8]:
len(paths)

24459

In [9]:
NUM_WORKERS = 4
CLASSES = sorted(os.listdir(AUDIO_PATH))

for dir_ in tqdm(CLASSES):
    _ = os.makedirs('/content/train_audio_sep_5sec_ogg/' + dir_, exist_ok=True)
#_ = Parallel(n_jobs=NUM_WORKERS)(delayed(save_full)(AUDIO_PATH) for AUDIO_PATH in tqdm(paths[step * i:step * (i+1)]))
_ = Parallel(n_jobs=NUM_WORKERS)(delayed(save_ogg_full)(AUDIO_PATH) for AUDIO_PATH in tqdm(paths))

100%|██████████| 24459/24459 [1:25:19<00:00,  4.78it/s]


In [10]:
len(glob.glob('/content/train_audio_sep_5sec_ogg/*/*'))

194213

In [ ]:
!zip -r -q train_audio_sep_5sec_ogg.zip ./train_audio_sep_5sec_ogg/

In [ ]:
!cp train_audio_sep_5sec_ogg.zip /content/drive/MyDrive/ColabNotebooks/BirdCLEF2024/data/